<a href="https://colab.research.google.com/github/wily037/Ameyo-data-minds/blob/main/Tai_lab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

code = """
from fastapi import FastAPI, UploadFile, File
from fastapi.middleware.cors import CORSMiddleware
import pandas as pd
import io

app = FastAPI()

app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

@app.post("/analyze/")
async def analyze_csv(file: UploadFile = File(...)):
    try:
        contents = await file.read()
        df = pd.read_csv(io.StringIO(contents.decode("utf-8")))

        summary = {
            "columns": df.columns.tolist(),
            "shape": df.shape,
            "missing_values": df.isnull().sum().to_dict(),
            "mean": df.mean(numeric_only=True).to_dict(),
            "median": df.median(numeric_only=True).to_dict(),
            "head": df.head().to_dict(orient="records")
        }

        return {"status": "success", "summary": summary}

    except Exception as e:
        return {"status": "error", "message": str(e)}
"""

with open("main.py", "w") as f:
    f.write(code)
from fastapi import FastAPI, UploadFile, File
from fastapi.middleware.cors import CORSMiddleware
import pandas as pd
import io

app = FastAPI()

# Enable CORS (so frontend can call the API later)
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],  # Change to specific domain in production
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

@app.post("/analyze/")
async def analyze_csv(file: UploadFile = File(...)):
    try:
        contents = await file.read()
        df = pd.read_csv(io.StringIO(contents.decode("utf-8")))

        # Summary statistics
        summary = {
            "columns": df.columns.tolist(),
            "shape": df.shape,
            "missing_values": df.isnull().sum().to_dict(),
            "mean": df.mean(numeric_only=True).to_dict(),
            "median": df.median(numeric_only=True).to_dict(),
            "head": df.head().to_dict(orient="records")
        }

        return {"status": "success", "summary": summary}

    except Exception as e:
        return {"status": "error", "message": str(e)}

ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-31' coro=<Server.serve() done, defined at /usr/local/lib/python3.11/dist-packages/uvicorn/server.py:69> exception=SystemExit(1)>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/uvicorn/server.py", line 164, in startup
    server = await loop.create_server(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/asyncio/base_events.py", line 1536, in create_server
    raise OSError(err.errno, msg) from None
OSError: [Errno 98] error while attempting to bind on address ('0.0.0.0', 8000): address already in use

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/lib/python3.11/threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.11/threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
  File "/tmp/ipython-input-26-1877427048.py", line 15, in

In [ ]:
!pip install fastapi uvicorn nest-asyncio pyngrok pandas

In [ ]:

from pyngrok import ngrok

# Paste your auth token below between the quotes
ngrok.set_auth_token("30KcWzC7n4I7sWgVgLAQLseeJGB_5QwBLdsMuQTF2jFkvmLbX")

In [ ]:

from fastapi import File, UploadFile
import pandas as pd
import io

@app.post("/upload_csv/")
async def upload_csv(file: UploadFile = File(...)):
    contents = await file.read()
    df = pd.read_csv(io.StringIO(contents.decode("utf-8")))
    return {
        "filename": file.filename,
        "columns": df.columns.tolist(),
        "rows": len(df)
    }

In [ ]:

from pyngrok import ngrok
ngrok.kill()

In [ ]:
import nest_asyncio
import threading
from pyngrok import ngrok
import uvicorn

# Allow async code in Colab
nest_asyncio.apply()

# Set up the public URL
public_url = ngrok.connect(8000)
print(f"🔗 Your Public API: {public_url}/docs")

# Function to run server
def run():
    uvicorn.run("main:app", host="0.0.0.0", port=8000)

# Start server in background
thread = threading.Thread(target=run)
thread.start()

🔗 Your Public API: NgrokTunnel: "https://1e22cd5d052f.ngrok-free.app" -> "http://localhost:8000"/docs
